# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [ ]:
# ! pip install -q schedule pytest 
# установка библиотек, если ещё не

In [ ]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
from datetime import datetime, timedelta  
import requests
import schedule
from bs4 import BeautifulSoup

from tqdm.notebook import tqdm  
import json

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [ ]:
 def get_book_data(book_url: str) -> dict:
    """
    Функция парсит страницу с данными по книге.

    Args:
       book_url (str): Ссылка на страницу.
       
    Returns:
       res (dict): Словарь с полученной информацией.
    """
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    resp = requests.get(book_url, timeout=10)
    soup = BeautifulSoup(resp.text, 'html.parser')

    d_1 = {
        "site name": soup.find('title').text.strip().split(' | ')[1],
        
        "title": soup.find('title').text.strip().split(' | ')[0]
    }

    d_2 = {}
    # парсинг метаданных
    for tag in soup.find_all("meta"):
        keys = tag.attrs.keys()
        for k in keys:
            if k != 'content':
                d_2[tag[k]] = tag['content'].strip()
            else:
                continue

    d_3 = {}
    # парсинг таблицы
    for t in soup.find_all("tr"):
        d_3[t.find('th').get_text()] = t.find('td').get_text()

     
    res = {**d_1, **d_2, **d_3}

     
    return res
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
resp = get_book_data(book_url)

In [ ]:
resp

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [ ]:
def scrape_books(is_save : bool = True, paige_count : int = None) -> list:
    """
    Функция парсит все страницы с книгами.

    Args:
       is_save (bool): флаг на сохранение в books_data.txt.
       paige_count (int): количество страниц для парсинга
       
    Returns:
       res (list): Список словарей по каждой книге со всех страниц.
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    print(f"ВЫПОЛНЯЮ ПАРСИНГ! {datetime.now().strftime('%H:%M:%S')}")
    
    if paige_count:
        print ("paige count: ", paige_count)
    else:
        main = "https://books.toscrape.com/index.html"
        r = requests.get(main, timeout=10)
        s = BeautifulSoup(r.text, 'html.parser')
        paige_count = int(s.find_all("li", {"class":"current"})[0].text.strip().split(' ')[-1])
        print ("paige count: ", paige_count)

    lst = []
    for paige in tqdm(range(1, paige_count + 1)):
        paige = f"http://books.toscrape.com/catalogue/page-{paige}.html"
        response = requests.get(paige, timeout=10)
        sp = BeautifulSoup(response.text, 'html.parser')
        
        for h in sp.find_all("h3"):
            book = h.find('a')['href']
            book_url = f"https://books.toscrape.com/catalogue/{book}"
            lst.append(get_book_data(book_url))

    if is_save:
        with open('books.txt', 'w', encoding='utf-8') as f:
            for book in lst:
                f.write(json.dumps(book, ensure_ascii=False) + '\n')

    return lst
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
# Проверка работоспособности функции
res = scrape_books(is_save=True, paige_count = 2) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
# Расписание
schedule.every().day.at("19:00").do(your_scraping_function)

# Для теста - запуск через 1 минуту
# test_time = (datetime.now() + timedelta(minutes=1)).strftime("%H:%M")
# schedule.every().day.at(test_time).do(
#     scrape_books,
#     is_save=True, 
#     paige_count=2
# )
print(f"Запланирован парсинг на {test_time}")

print("Скрипт работает...")

while True:
    schedule.run_pending()
    print(f"Проверка расписания... {datetime.now().strftime('%H:%M:%S')}")
    time.sleep(30)

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [ ]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте

!../books_venv/bin/python3 -m pytest ../tests/test_scraper.py

## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```